In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#%cd /content/drive/My Drive/Thesis/Data/NER

In [ ]:
!pip install transformers seqeval[gpu]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=e0578cc2343c11bd08fb2ba5f9633cf12ac35b627d4035777958b0e3a10888c6
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
import pandas as pd
import ast

df = pd.read_csv("AnnotatedDataProcessedCleaned.csv")
#Since rows are stings not 2d list
df['entities'] = df['entities'].apply(ast.literal_eval)

In [ ]:
df

,Unnamed: 0,text,entities
0,0,1st family: a healthy queen observed workers a...,"[[14, 21, HEALTHSTATUS], [22, 27, QUEEN], [37,..."
1,1,2nd family: queenright with an extensive brood...,"[[12, 22, QUEEN], [55, 62, WORKER], [94, 99, B..."
2,2,3rd family: Queen’s presence confirmed strong ...,"[[12, 19, QUEEN], [88, 96, BEEPRODUCT], [102, ..."
3,3,4th family: Queen is laying well the worker po...,"[[12, 17, QUEEN], [37, 43, WORKER], [44, 62, H..."
4,4,5th family: Queen is healthy drone population ...,"[[12, 17, QUEEN], [21, 28, HEALTHSTATUS], [29,..."
...,...,...,...
553,553,"700th family: queen presence weak, workers exh...","[[14, 19, QUEEN], [20, 33, HEALTHSTATUS], [35,..."
554,554,"701st family: queen observed lethargic, worker...","[[14, 19, QUEEN], [29, 38, HEALTHSTATUS], [40,..."
555,555,"702nd family: queenless situation confirmed, w...","[[14, 23, QUEEN], [34, 43, HEALTHSTATUS], [45,..."
556,556,"703rd family: queenright uncertain, worker act...","[[14, 24, QUEEN], [25, 34, HEALTHSTATUS], [36,..."


In [ ]:
def convert_to_bio_format(text, entities, tokenizer):
    encoding = tokenizer(text, return_offsets_mapping=True)
    tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
    token_offsets = encoding['offset_mapping']
    bio_tags = ['O'] * len(tokens)

    for start, end, label in entities:
        entity_start = True
        for idx, (token_start, token_end) in enumerate(token_offsets):
            if token_start is None or token_end is None:
                continue  # Skip special tokens like [CLS] and [SEP]
            if token_start >= start and token_end <= end:
                if entity_start:
                    bio_tags[idx] = 'B-' + label
                    entity_start = False
                else:
                    bio_tags[idx] = 'I-' + label

    return tokens, bio_tags

In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

bio_data = []

for index, row in df.iterrows():
    text = row['text']
    entities = row['entities']
    tokens, bio_tags = convert_to_bio_format(text, entities, tokenizer)
    bio_data.append((tokens, bio_tags))

# Convert the result to a DataFrame for better visualization
bio_df = pd.DataFrame(bio_data, columns=['tokens', 'bio_tags'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
bio_df

,tokens,bio_tags
0,"[[CLS], 1st, family, :, a, healthy, queen, obs...","[O, O, O, O, O, B-HEALTHSTATUS, B-QUEEN, O, B-..."
1,"[[CLS], 2nd, family, :, queen, ##right, with, ...","[O, O, O, O, B-QUEEN, I-QUEEN, O, O, O, O, O, ..."
2,"[[CLS], 3rd, family, :, Queen, ’, s, presence,...","[O, O, O, O, B-QUEEN, I-QUEEN, I-QUEEN, O, O, ..."
3,"[[CLS], 4th, family, :, Queen, is, laying, wel...","[O, O, O, O, B-QUEEN, O, O, O, O, B-WORKER, B-..."
4,"[[CLS], 5th, family, :, Queen, is, healthy, dr...","[O, O, O, O, B-QUEEN, O, B-HEALTHSTATUS, B-DRO..."
...,...,...
553,"[[CLS], 700, ##th, family, :, queen, presence,...","[O, O, O, O, O, B-QUEEN, B-HEALTHSTATUS, I-HEA..."
554,"[[CLS], 70, ##1, ##st, family, :, queen, obser...","[O, O, O, O, O, O, B-QUEEN, O, B-HEALTHSTATUS,..."
555,"[[CLS], 70, ##2, ##nd, family, :, queen, ##les...","[O, O, O, O, O, O, B-QUEEN, I-QUEEN, O, B-HEAL..."
556,"[[CLS], 70, ##3, ##rd, family, :, queen, ##rig...","[O, O, O, O, O, O, B-QUEEN, I-QUEEN, B-HEALTHS..."


In [ ]:
bio_df['sentence'] = df["text"]

In [ ]:
bio_df['sentence'].str.strip() #Whitespace removal

0      1st family: a healthy queen observed workers a...
1      2nd family: queenright with an extensive brood...
2      3rd family: Queen’s presence confirmed strong ...
3      4th family: Queen is laying well the worker po...
4      5th family: Queen is healthy drone population ...
                             ...                        
553    700th family: queen presence weak, workers exh...
554    701st family: queen observed lethargic, worker...
555    702nd family: queenless situation confirmed, w...
556    703rd family: queenright uncertain, worker act...
557    704th family: queen observed lethargic, worker...
Name: sentence, Length: 558, dtype: object

In [ ]:
bio_df.count()

tokens      558
bio_tags    558
sentence    558
dtype: int64

In [ ]:
bio_df

,tokens,bio_tags,sentence
0,"[[CLS], 1st, family, :, a, healthy, queen, obs...","[O, O, O, O, O, B-HEALTHSTATUS, B-QUEEN, O, B-...",1st family: a healthy queen observed workers a...
1,"[[CLS], 2nd, family, :, queen, ##right, with, ...","[O, O, O, O, B-QUEEN, I-QUEEN, O, O, O, O, O, ...",2nd family: queenright with an extensive brood...
2,"[[CLS], 3rd, family, :, Queen, ’, s, presence,...","[O, O, O, O, B-QUEEN, I-QUEEN, I-QUEEN, O, O, ...",3rd family: Queen’s presence confirmed strong ...
3,"[[CLS], 4th, family, :, Queen, is, laying, wel...","[O, O, O, O, B-QUEEN, O, O, O, O, B-WORKER, B-...",4th family: Queen is laying well the worker po...
4,"[[CLS], 5th, family, :, Queen, is, healthy, dr...","[O, O, O, O, B-QUEEN, O, B-HEALTHSTATUS, B-DRO...",5th family: Queen is healthy drone population ...
...,...,...,...
553,"[[CLS], 700, ##th, family, :, queen, presence,...","[O, O, O, O, O, B-QUEEN, B-HEALTHSTATUS, I-HEA...","700th family: queen presence weak, workers exh..."
554,"[[CLS], 70, ##1, ##st, family, :, queen, obser...","[O, O, O, O, O, O, B-QUEEN, O, B-HEALTHSTATUS,...","701st family: queen observed lethargic, worker..."
555,"[[CLS], 70, ##2, ##nd, family, :, queen, ##les...","[O, O, O, O, O, O, B-QUEEN, I-QUEEN, O, B-HEAL...","702nd family: queenless situation confirmed, w..."
556,"[[CLS], 70, ##3, ##rd, family, :, queen, ##rig...","[O, O, O, O, O, O, B-QUEEN, I-QUEEN, B-HEALTHS...","703rd family: queenright uncertain, worker act..."


In [ ]:
from collections import Counter

all_tags = [tag for tags in bio_df['bio_tags'] for tag in tags]
tag_counts = Counter(all_tags)

# Summarize the counts for each class (ignoring the 'O' tag)
class_counts = Counter()
for tag, count in tag_counts.items():
        class_counts[tag[2:]] += count  # tag[2:] to remove the B- or I- prefix

print("Tag Counts:", tag_counts)
print("Class Counts:", class_counts)

Tag Counts: Counter({'O': 9696, 'I-HONEYFRAMES': 1289, 'B-HEALTHSTATUS': 1069, 'I-HEALTHSTATUS': 935, 'I-EGGCOUNT': 681, 'B-HONEYFRAMES': 593, 'B-WORKER': 517, 'B-QUEEN': 481, 'B-BEEPRODUCT': 426, 'B-EGGCOUNT': 387, 'I-BEEPRODUCT': 366, 'B-DRONE': 186, 'B-FORAGE': 145, 'I-FORAGE': 142, 'I-DRONE': 127, 'B-BEE': 63, 'I-QUEEN': 57, 'I-BEE': 18, 'I-WORKER': 5})
Class Counts: Counter({'': 9696, 'HEALTHSTATUS': 2004, 'HONEYFRAMES': 1882, 'EGGCOUNT': 1068, 'BEEPRODUCT': 792, 'QUEEN': 538, 'WORKER': 522, 'DRONE': 313, 'FORAGE': 287, 'BEE': 81})


In [ ]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(bio_df, test_size=0.2, random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [ ]:
train_labels = train_df['bio_tags'].explode().tolist()
test_labels = test_df['bio_tags'].explode().tolist()
valid_labels = valid_df['bio_tags'].explode().tolist()

In [ ]:
train_labels

['O',
 'O',
 'O',
 'O',
 'O',
 'B-QUEEN',
 'O',
 'O',
 'B-WORKER',
 'B-HEALTHSTATUS',
 'B-DRONE',
 'I-DRONE',
 'B-EGGCOUNT',
 'I-EGGCOUNT',
 'O',
 'B-HONEYFRAMES',
 'I-HONEYFRAMES',
 'B-HEALTHSTATUS',
 'O',
 'O',
 'B-BEEPRODUCT',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-QUEEN',
 'B-HEALTHSTATUS',
 'B-WORKER',
 'O',
 'B-DRONE',
 'I-DRONE',
 'B-HEALTHSTATUS',
 'I-HEALTHSTATUS',
 'I-HEALTHSTATUS',
 'I-HEALTHSTATUS',
 'O',
 'O',
 'B-HONEYFRAMES',
 'I-HONEYFRAMES',
 'I-HONEYFRAMES',
 'I-HONEYFRAMES',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-QUEEN',
 'B-HEALTHSTATUS',
 'B-WORKER',
 'I-WORKER',
 'I-WORKER',
 'B-WORKER',
 'B-HEALTHSTATUS',
 'B-DRONE',
 'I-DRONE',
 'O',
 'B-EGGCOUNT',
 'I-EGGCOUNT',
 'I-EGGCOUNT',
 'B-HONEYFRAMES',
 'I-HONEYFRAMES',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-QUEEN',
 'B-HEALTHSTATUS',
 'B-HEALTHSTATUS',
 'B-WORKER',
 'O',
 'O',
 'B-DRONE',
 'I-DRONE',
 'O',
 'B-EGGCOUNT',
 'I-EGGCOUNT',
 'B-HONEYFRAMES',
 'I-HONEYFRAMES',
 'I-HONEYFRAME

In [ ]:
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)
valid_df.to_csv('valid.csv',index=False)

In [ ]:
from transformers import BertTokenizerFast
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 20
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

In [ ]:
class CustomNERDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, label_to_id):
        self.data = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label_to_id = label_to_id

    def __getitem__(self, index):
        tokens = self.data.iloc[index]['tokens']
        labels = self.data.iloc[index]['bio_tags']

        # Truncate the sequence if it exceeds the maximum length
        if len(tokens) > self.max_len - 2:  # Account for [CLS] and [SEP]
            tokens = tokens[:self.max_len - 2]
            labels = labels[:self.max_len - 2]

        # Add [CLS] and [SEP] tokens
        #tokens = ['[CLS]'] + tokens + ['[SEP]']
        #labels = ['O'] + labels + ['O']

        # Pad the sequence if it is shorter than the maximum length
        pad_length = self.max_len - len(tokens)
        tokens.extend(['[PAD]'] * pad_length)
        labels.extend(['O'] * pad_length)

        # Create attention masks
        attn_mask = [1 if token != '[PAD]' else 0 for token in tokens]

        # Convert tokens and labels to their respective IDs
        ids = self.tokenizer.convert_tokens_to_ids(tokens)
        label_ids = [self.label_to_id.get(label, self.label_to_id['O']) for label in labels]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(attn_mask, dtype=torch.long),
            'targets': torch.tensor(label_ids, dtype=torch.long)
        }

    def __len__(self):
        return len(self.data)


In [ ]:
from transformers import BertTokenizerFast


label_to_id = {'O': 0 ,
               'I-HONEYFRAMES': 1,
               'B-HEALTHSTATUS': 2,
               'I-HEALTHSTATUS': 3,
               'I-EGGCOUNT': 4,
               'B-HONEYFRAMES': 5,
               'B-WORKER': 6,
               'B-QUEEN': 7,
               'B-BEEPRODUCT': 8,
               'B-EGGCOUNT': 9,
               'I-BEEPRODUCT': 10,
               'B-DRONE': 11,
               'B-FORAGE': 12,
               'I-FORAGE': 13,
               'I-DRONE': 14,
               'B-BEE': 15,
               'I-QUEEN': 16,
               'I-BEE': 17,
               'I-WORKER': 18
}

id_to_label = {id_: label for label, id_ in label_to_id.items()}


tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

train_df = CustomNERDataset(train_df, tokenizer=tokenizer, max_len=128, label_to_id=label_to_id)
test_df = CustomNERDataset(dataframe=test_df, tokenizer=tokenizer, max_len=128, label_to_id=label_to_id)
valid_df = CustomNERDataset(dataframe=valid_df, tokenizer=tokenizer, max_len=128, label_to_id=label_to_id)


In [ ]:
train_df[0]

{'ids': tensor([  101, 22690,  1582,  1266,   131,  2454, 18652,  5116,  3239,  8071,
         22020,  1116,  6471, 13504,  1105,  2551, 11382,  3777, 23164,  1114,
          8531,   119,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,  

In [ ]:
for token, label in zip(tokenizer.convert_ids_to_tokens(train_df[0]["ids"]), train_df[0]["targets"]):
  print('{0:10}  {1}'.format(token, label))

[CLS]       0
199         0
##th        0
family      0
:           0
Queen       7
vibrant     0
busy        0
workers     6
healthy     2
drone       11
##s         14
eggs        9
abundant    4
and         0
nine        5
frames      1
heavily     2
laden       0
with        0
honey       8
.           0
[SEP]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

valid_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
               'num_workers': 0
               }

training_loader = DataLoader(train_df, **train_params)
validating_loader = DataLoader(valid_df, **valid_params)
testing_loader = DataLoader(test_df, **valid_params)

In [ ]:
assert all(label_id in range(len(label_to_id)) for label_id in label_to_id.values()), "Label ID out of range in label_to_id mapping."

In [ ]:
model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(label_to_id))
model.to(device)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [ ]:
inputs = train_df[2]
input_ids = inputs["ids"].unsqueeze(0)
attention_mask = inputs["mask"].unsqueeze(0)
labels = inputs["targets"].unsqueeze(0)

print(input_ids)
print(attention_mask)
print(labels)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor([[  101, 17881,  3276,  1266,   131,  2454, 13909,  1107, 27650, 17401,
          3239, 12373, 22020,  1116,  1344,  9069,  1707,  1105,  1565, 11382,
          3106, 14510,   119,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

tensor(3.1011, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 19])

In [ ]:
initial_loss.item()

3.1011335849761963

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [ ]:
# Extract unique labels from your dataset
unique_labels_in_dataset = set(label for labels in train_df['bio_tags'] for label in labels)

# Print unique labels to check
print("Unique labels in dataset:", len( unique_labels_in_dataset))
print("Labels in label_to_id:", len(label_to_id.keys()))


Unique labels in dataset: 19
Labels in label_to_id: 19


In [ ]:
import torch
import numpy as np
import torch.nn as nn
from collections import defaultdict

def calculate_class_weights(labels, label_to_id):
    label_counts = defaultdict(int)
    for label_list in labels:
        for label in label_list:
            if label in label_to_id:  # Ensure label exists in label_to_id
                label_counts[label_to_id[label]] += 1

    total_counts = sum(label_counts.values())
    class_weights = {label: total_counts / count for label, count in label_counts.items()}

    # Normalize weights
    max_weight = max(class_weights.values())
    class_weights = {label: weight / max_weight for label, weight in class_weights.items()}

    return class_weights

# Ensure labels are in list format
#train_labels = train_df['bio_tags'].tolist()

# Flatten train_labels list
flattened_train_labels = [label for sublist in train_labels for label in sublist]

# Calculate class weights
class_weights = calculate_class_weights(flattened_train_labels, label_to_id)

# Convert weights to tensor
class_weights_tensor = torch.tensor([class_weights[label] if label in class_weights else 1.0 for label in range(len(label_to_id))], device=device)

# Define loss function with class weights
loss_fn = nn.CrossEntropyLoss(weight=class_weights_tensor)

print("Class weights tensor:", class_weights_tensor)


def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    model.train()

    for idx, batch in enumerate(training_loader):
        ids = batch['ids'].to(device, dtype=torch.long)
        mask = batch['mask'].to(device, dtype=torch.long)
        labels = batch['targets'].to(device, dtype=torch.long)

        ids = ids.to(device)
        mask = mask.to(device)
        labels = labels.to(device)

        outputs = model(input_ids=ids, attention_mask=mask)
        loss = loss_fn(outputs.logits.view(-1, model.num_labels), labels.view(-1))
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)

        if idx % 100 == 0:
            loss_step = tr_loss / nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        flattened_targets = labels.view(-1)
        active_logits = outputs.logits.view(-1, model.num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1)

        active_accuracy = labels.view(-1) != -100
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels.cpu().numpy())
        tr_preds.extend(predictions.cpu().numpy())

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=MAX_GRAD_NORM)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps

    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

    return tr_labels, tr_preds

# Example usage
all_tr_labels, all_tr_preds = [], []
for epoch in range(1, EPOCHS + 1):
    tr_labels, tr_preds = train(epoch)
    all_tr_labels.extend(tr_labels)
    all_tr_preds.extend(tr_preds)


Class weights tensor: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1.], device='cuda:0')
Training loss per 100 training steps: 3.104374885559082
Training loss epoch: 1.3513737746647425
Training accuracy epoch: 0.7840252311862245
Training loss per 100 training steps: 0.5392141938209534
Training loss epoch: 0.4291270800999233
Training accuracy epoch: 0.8962751116071429
Training loss per 100 training steps: 0.3547065854072571
Training loss epoch: 0.32629328433956417
Training accuracy epoch: 0.9062151227678571
Training loss per 100 training steps: 0.2853867709636688
Training loss epoch: 0.2590248504919665
Training accuracy epoch: 0.9273781289859694
Training loss per 100 training steps: 0.22580847144126892
Training loss epoch: 0.20389651081391744
Training accuracy epoch: 0.9441864636479592
Training loss per 100 training steps: 0.17480716109275818
Training loss epoch: 0.16593273941959655
Training accuracy epoch: 0.9532420878507653
Training loss per 

NameError: name 'classification_report' is not defined

In [ ]:
# Generate classification report at the end of training
from sklearn.metrics import classification_report
report = classification_report(all_tr_labels, all_tr_preds, target_names=list(label_to_id.keys()), digits=2)
print(report)

                precision    recall  f1-score   support

             O       0.97      0.99      0.98   1022480
 I-HONEYFRAMES       0.82      0.81      0.82     21160
B-HEALTHSTATUS       0.60      0.46      0.52     17080
I-HEALTHSTATUS       0.64      0.44      0.53     14660
    I-EGGCOUNT       0.76      0.54      0.63     10820
 B-HONEYFRAMES       0.89      0.72      0.80      9580
      B-WORKER       0.97      0.83      0.90      8180
       B-QUEEN       0.83      0.83      0.83      7560
  B-BEEPRODUCT       0.92      0.75      0.83      6620
    B-EGGCOUNT       0.74      0.59      0.66      6200
  I-BEEPRODUCT       0.95      0.83      0.89      5620
       B-DRONE       0.72      0.78      0.75      2960
      B-FORAGE       0.95      0.68      0.80      2320
      I-FORAGE       0.97      0.78      0.86      2260
       I-DRONE       0.33      0.71      0.45      2100
         B-BEE       0.92      0.38      0.54      1000
       I-QUEEN       0.30      0.38      0.34  

In [ ]:
pip install nervaluate

In [ ]:
def convert_labels(numerical_labels):
    return [id_to_label[label] for label in numerical_labels]

# Converted labels and predictions
all_tr_labels_converted = convert_labels(all_tr_labels)
all_tr_preds_converted = convert_labels(all_tr_preds)

In [ ]:
sequence_length = 5

def split_into_sequences(labels, sequence_length):
    return [labels[i:i + sequence_length] for i in range(0, len(labels), sequence_length)]

true_sequences = split_into_sequences(all_tr_labels_converted, sequence_length)
pred_sequences = split_into_sequences(all_tr_preds_converted, sequence_length)


In [ ]:
true_sequences

[['O', 'O', 'O', 'O', 'O'],
 ['B-QUEEN', 'O', 'O', 'O', 'O'],
 ['B-WORKER', 'B-HEALTHSTATUS', 'O', 'O', 'O'],
 ['B-HEALTHSTATUS', 'B-BEEPRODUCT', 'I-BEEPRODUCT', 'I-BEEPRODUCT', 'O'],
 ['B-BEEPRODUCT', 'I-BEEPRODUCT', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'B-BEEPRODUCT'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'B-BEE', 'O'],
 ['O', 'O', 'O', 'B-WORKER', 'B-HEALTHSTATUS'],
 ['O', 'B-HEALTHSTATUS', 'I-HEALTHSTATUS', 'O', 'O'],
 ['O', 'O', 'B-HONEYFRAMES', 'I-H

In [ ]:
from nervaluate import Evaluator

evaluator = Evaluator(all_tr_labels_converted, all_tr_preds_converted, tags=list(id_to_label.values())[1:], loader='list')

results, results_by_tag = evaluator.evaluate()

In [ ]:
results

NameError: name 'results' is not defined

In [ ]:
from collections import defaultdict

def flip_nested_dict(dd):
    result = defaultdict(dict)
    for k1, d in dd.items():
        for k2, v in d.items():
            result[k2][k1] = v
    return dict(result)

In [ ]:
def valid(model, validating_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(validating_loader):

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            labels = batch['targets'].to(device, dtype = torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += outputs[0].item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = outputs[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)

            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(labels)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [id_to_label[id.item()] for id in eval_labels]
    predictions = [id_to_label[id.item()] for id in eval_preds]

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [ ]:
labels, predictions = valid(model, validating_loader)

Validation loss per 100 evaluation steps: 0.09766791760921478
Validation Loss: 0.09612248837947845
Validation Accuracy: 0.9705810546875


In [ ]:
report_valid = classification_report(labels,predictions, digits=2)
print(report_valid)

                precision    recall  f1-score   support

         B-BEE       1.00      0.38      0.55         8
  B-BEEPRODUCT       0.89      0.94      0.92        52
       B-DRONE       0.65      1.00      0.79        15
    B-EGGCOUNT       0.82      0.90      0.86        40
      B-FORAGE       1.00      1.00      1.00        16
B-HEALTHSTATUS       0.64      0.81      0.72       105
 B-HONEYFRAMES       0.90      0.90      0.90        58
       B-QUEEN       0.96      0.94      0.95        52
      B-WORKER       1.00      1.00      1.00        54
         I-BEE       0.00      0.00      0.00         3
  I-BEEPRODUCT       0.94      1.00      0.97        48
       I-DRONE       0.59      1.00      0.74        10
    I-EGGCOUNT       0.85      0.76      0.80        74
      I-FORAGE       1.00      1.00      1.00        16
I-HEALTHSTATUS       0.59      0.72      0.65        86
 I-HONEYFRAMES       0.94      0.93      0.94       123
       I-QUEEN       0.86      0.60      0.71  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.13798899948596954
Validation Loss: 0.1024266080930829
Validation Accuracy: 0.9686279296875


In [ ]:
from sklearn.metrics import classification_report
report_test = classification_report(labels, predictions)
print(report_test)

                precision    recall  f1-score   support

         B-BEE       1.00      0.60      0.75         5
  B-BEEPRODUCT       0.91      0.93      0.92        43
       B-DRONE       0.85      1.00      0.92        23
    B-EGGCOUNT       0.69      0.84      0.76        37
      B-FORAGE       0.86      0.92      0.89        13
B-HEALTHSTATUS       0.56      0.70      0.62       110
 B-HONEYFRAMES       0.86      0.88      0.87        56
       B-QUEEN       1.00      1.00      1.00        51
      B-WORKER       1.00      0.98      0.99        54
         I-BEE       0.00      0.00      0.00         1
  I-BEEPRODUCT       0.97      0.86      0.91        37
       I-DRONE       0.75      1.00      0.86        12
    I-EGGCOUNT       0.68      0.83      0.75        66
      I-FORAGE       0.75      0.92      0.83        13
I-HEALTHSTATUS       0.70      0.69      0.69       116
 I-HONEYFRAMES       0.92      0.93      0.92       108
       I-QUEEN       1.00      0.71      0.83  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Define the save directory
save_directory = "./fine_tuned_model"

# Save the model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")
